In [1]:
import pandas as pd
import numpy as np
import cdb
from sklearn.preprocessing import OneHotEncoder
import timeit
import xgboost as xgb

In [2]:
store = cdb.use('mrg')
with store.open("/e737253/train_2.csv",'rb') as f:
    train1  = pd.read_csv(f)

In [3]:
missing = train1.isnull()
train1 = train1.fillna(0)
train1.iloc[:,1:] = train1.iloc[:,1:].astype(np.float32)
n = train1.shape[0]

In [4]:
a = train1.Page.str.split('mediawiki.org|wikimedia.org|wikipedia.org',expand=True)
b = np.where(a.loc[:,2].isnull(),a.loc[:,1],a.loc[:,2])
_,index_array = np.unique(b,return_inverse=True) # access & agent
website_array = [train1.Page.str.contains(web).values.astype(np.int8) \
                 for web in ['mediawiki.org','wikimedia.org','wikipedia.org']] # website
lang_array = train1.Page.str.extract('([a-z]{2}).wikipedia').fillna('nan').values.flatten()
_,lang_array = np.unique(lang_array,return_inverse=True) # language

/apps/conda/envs/ml_py36.18.06.r2/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [5]:
Page_X = \
np.concatenate([OneHotEncoder(dtype=np.float32,sparse=False).fit_transform(lang_array[:,np.newaxis]),\
                OneHotEncoder(dtype=np.float32,sparse=False).fit_transform(index_array[:,np.newaxis]),\
                np.stack(website_array,1)],1)

In [6]:
date_index=pd.to_datetime(train1.columns[1:]).to_series().reset_index(drop=True)
def extract_date_info(timestamp,mss):
    return np.array([mss, timestamp.weekday(), timestamp.month, timestamp.day, \
                     timestamp.dayofyear,timestamp.year])

In [7]:
d = extract_date_info(date_index[3],3).shape[0]

In [8]:
max_ = lambda df,start,interval: np.max(df.iloc[:,start-interval:start].values,1)
min_ = lambda df,start,interval: np.min(df.iloc[:,start-interval:start].values,1)
std_ = lambda df,start,interval: np.std(df.iloc[:,start-interval:start].values,1)
mean_ = lambda df,start,interval: np.mean(df.iloc[:,start-interval:start].values,1)
growth_ = lambda df,start,interval: np.mean(df.iloc[:,start-interval:start],1) -\
                                    np.mean(df.iloc[:,start-2*interval:start-interval],1)

fun_list = [max_,min_,std_,mean_,growth_]
interval_list = [14,30,60,120]
T = train1.shape[1]

In [9]:
#margain_ = np.median(train1.iloc[:,1:].values,1)
#margain_ = np.where(margain_==0,-20,np.log(margain_))

In [159]:
downSampleRate = 0.5
j = T - 64
Lookback_X = np.stack([fun(train1,j,interval) for fun in fun_list for interval in interval_list],1)
testData = []
testMargain = []
for i in range(j+2,j+64):
    temp = np.concatenate([train1.iloc[:,i].values[:,np.newaxis],\
                           Page_X,\
                           np.broadcast_to(extract_date_info(date_index[i-1],i-j),(n,d)),\
                           Lookback_X,\
                           train1.iloc[:,j-1:j].values,\
                           train1.iloc[:,i-90:i-89].values,\
                           train1.iloc[:,i-120:i-119].values,\
                           train1.iloc[:,i-180:i-179].values,\
                           train1.iloc[:,i-365:i-364].values],1).astype(np.float32)
    nonmissing_index = np.logical_not(missing.iloc[:,i].values)
    random_index = np.random.rand(nonmissing_index.sum())<downSampleRate
    testData.append(temp[nonmissing_index][random_index])
    #testMargain.append(margain_[nonmissing_index][random_index])
    testMargain.append(train1.iloc[:,j-1].values[nonmissing_index][random_index])

In [160]:
testData = np.concatenate(testData,0)
testMargain = np.concatenate(testMargain,0)

In [171]:
downSampleRate = 0.1
trainData = []
trainMargain = []
for j in range(T-65*2,350,-70):
    Lookback_X = np.stack([fun(train1,j,interval) for fun in fun_list for interval in interval_list],1)
    for i in range(j+2,j+64):
        temp = np.concatenate([train1.iloc[:,i].values[:,np.newaxis],\
                               Page_X,\
                               np.broadcast_to(extract_date_info(date_index[i-1],i-j),(n,d)),\
                               Lookback_X,\
                               train1.iloc[:,j-1:j].values,\
                               train1.iloc[:,i-90:i-89].values,\
                               train1.iloc[:,i-120:i-119].values,\
                               train1.iloc[:,i-180:i-179].values,\
                               train1.iloc[:,i-365:i-364].values],1).astype(np.float32)
        nonmissing_index = np.logical_not(missing.iloc[:,i].values)
        random_index = np.random.rand(nonmissing_index.sum())<downSampleRate
        trainData.append(temp[nonmissing_index][random_index])
        #trainMargain.append(margain_[nonmissing_index][random_index])
        trainMargain.append(train1.iloc[:,j-1].values[nonmissing_index][random_index])
trainData = np.concatenate(trainData,0)
trainMargain = np.concatenate(trainMargain,0)

In [172]:
# Data corruption when creating a a DMatrix with label being a non-contiguous ndarray #2554
trainData = np.ascontiguousarray(trainData)
trainMargain = np.ascontiguousarray(trainMargain)

In [183]:
eps = 1
floor = 0.1
neg_grad = -1.0

In [187]:
def SMAPE_train(preds, dtrain):
    y = dtrain.get_label()
    temp = (2*y+eps)/(preds+y+eps)**2
    grad = np.where(preds>y,temp,np.where(preds<0,neg_grad,-1*temp))
    #index_ = np.isfinite(grad)
    #non_index_ = np.logical_not(index_)
    #print(preds[non_index_],y[non_index_])
    hess = np.ones_like(grad)
    return grad, hess

def SMAPE_eval(preds, dtrain):
    y = dtrain.get_label()
    summ = np.abs(y) + np.abs(preds)
    return 'SMAPE', 200*np.mean(np.where(summ==0, 0, np.abs(y - preds) / summ))

In [176]:
dtrain = xgb.DMatrix(np.ascontiguousarray(trainData[:,1:]),label=np.ascontiguousarray(trainData[:,0]))
dtest = xgb.DMatrix(np.ascontiguousarray(testData[:,1:]),label=np.ascontiguousarray(testData[:,0]))

In [177]:
dtrain.set_base_margin(trainMargain)
dtest.set_base_margin(testMargain)
watchlist = [(dtest, 'eval'), (dtrain, 'train')]

In [185]:
num_round = 500
params =   {'eta': 1e-1,
            'max_depth': 6, 
            'max_bin': 32,
            'subsample': 0.05,
            #'colsample_bylevel': 0.5, 
            'min_child_weight':100,
            'tree_method':'hist',
            'nthread':8}

In [186]:
start_time = timeit.default_timer()
model_gbm = xgb.train(params, dtrain, 500, watchlist, \
                      verbose_eval=100,\
                      obj=SMAPE_train, feval=SMAPE_eval)
print("--- %s seconds ---" % (timeit.default_timer() - start_time))

/apps/conda/envs/ml_py36.18.06.r2/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: overflow encountered in exp
/apps/conda/envs/ml_py36.18.06.r2/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: overflow encountered in multiply
/apps/conda/envs/ml_py36.18.06.r2/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: overflow encountered in square
/apps/conda/envs/ml_py36.18.06.r2/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in true_divide
/apps/conda/envs/ml_py36.18.06.r2/lib/python3.6/site-packages/ipykernel/__main__.py:13: RuntimeWarning: invalid value encountered in less


[0]	eval-rmse:nan	train-rmse:nan	eval-SMAPE:nan	train-SMAPE:nan


/apps/conda/envs/ml_py36.18.06.r2/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in greater


KeyboardInterrupt: 

In [178]:
para_gen = {'eps':[np.random.choice,[0.1,1,5,10,50]],\
            'neg_grad':[np.random.choice,[-0.1,-1,-5,-10,-50]],\
            'max_depth':[np.random.choice,[4,8,12,16,24,32]],\
            'subsample':[np.random.choice,[0.001,0.0025,0.005,0.01,0.025,0.05]],\
            'colsample_bylevel':[np.random.choice,[0.5,0.75,1]],\
            'gamma':[np.random.choice,[0.1,1,10,100,1000]]}

In [179]:
def generate_(para_gen):
    return {key:item[0](item[1]) for key,item in para_gen.items()}

In [180]:
def RandomSearch(fun,para_gen,dtrain,dtest,iterations):
    # fun needs to have args Xtrain,ytrain,Xtest,ytest
    for _ in range(iterations):
        paras = generate_(para_gen)
        paras_data = paras.copy()
        paras_data['dtrain'] = dtrain
        paras_data['dtest'] = dtest
        paras['score'] = fun(**paras_data)
        print(paras)

In [181]:
def fun_xgb(dtrain,dtest,eps,neg_grad,max_depth,subsample,colsample_bylevel,gamma):
    
    def SMAPE_train(preds, dtrain):
        y = dtrain.get_label()
        temp = (2*y+eps)/(preds+y+eps)**2
        grad = np.where(preds>y,temp,np.where(preds<0,neg_grad,-1*temp))
        #index_ = np.isfinite(grad)
        #non_index_ = np.logical_not(index_)
        #print(preds[non_index_],y[non_index_])
        hess = np.ones_like(grad)
        return grad, hess
    
    num_round = 500
    params =   {'eta': 1e-1,
                'max_bin': 32,       
                'nthread':8,       
                'tree_method':'hist',                
                'max_depth': max_depth, 
                'subsample': subsample,
                'colsample_bylevel': colsample_bylevel, 
                'gamma': gamma
                }
    
    model_gbm = xgb.train(params, dtrain, num_round, obj=SMAPE_train,verbose_eval=False)
    return SMAPE_eval(model_gbm.predict(dtest), dtest)[1]

In [182]:
RandomSearch(fun_xgb,para_gen,dtrain,dtest,100)

{'eps': 10.0, 'neg_grad': -0.10000000000000001, 'max_depth': 12, 'subsample': 0.01, 'colsample_bylevel': 0.5, 'gamma': 0.10000000000000001, 'score': 47.334647178649902}
{'eps': 5.0, 'neg_grad': -50.0, 'max_depth': 8, 'subsample': 0.0025000000000000001, 'colsample_bylevel': 0.75, 'gamma': 10.0, 'score': 49.557363986968994}
{'eps': 0.10000000000000001, 'neg_grad': -10.0, 'max_depth': 32, 'subsample': 0.025000000000000001, 'colsample_bylevel': 0.5, 'gamma': 10.0, 'score': 47.02877402305603}
{'eps': 10.0, 'neg_grad': -1.0, 'max_depth': 8, 'subsample': 0.001, 'colsample_bylevel': 1.0, 'gamma': 0.10000000000000001, 'score': 47.854211926460266}
{'eps': 10.0, 'neg_grad': -5.0, 'max_depth': 32, 'subsample': 0.050000000000000003, 'colsample_bylevel': 1.0, 'gamma': 1000.0, 'score': 49.67702329158783}
{'eps': 10.0, 'neg_grad': -0.10000000000000001, 'max_depth': 24, 'subsample': 0.01, 'colsample_bylevel': 0.75, 'gamma': 0.10000000000000001, 'score': 47.330927848815918}
{'eps': 5.0, 'neg_grad': -10.

{'eps': 10.0, 'neg_grad': -0.10000000000000001, 'max_depth': 16, 'subsample': 0.0025000000000000001, 'colsample_bylevel': 0.75, 'gamma': 1.0, 'score': 49.652045965194702}
{'eps': 10.0, 'neg_grad': -5.0, 'max_depth': 24, 'subsample': 0.0025000000000000001, 'colsample_bylevel': 0.5, 'gamma': 1.0, 'score': 49.652045965194702}
{'eps': 1.0, 'neg_grad': -5.0, 'max_depth': 24, 'subsample': 0.01, 'colsample_bylevel': 1.0, 'gamma': 0.10000000000000001, 'score': 46.62441611289978}
{'eps': 5.0, 'neg_grad': -10.0, 'max_depth': 32, 'subsample': 0.0025000000000000001, 'colsample_bylevel': 0.75, 'gamma': 100.0, 'score': 49.557363986968994}
{'eps': 50.0, 'neg_grad': -1.0, 'max_depth': 16, 'subsample': 0.01, 'colsample_bylevel': 1.0, 'gamma': 1.0, 'score': 49.813753366470337}
{'eps': 5.0, 'neg_grad': -0.10000000000000001, 'max_depth': 12, 'subsample': 0.025000000000000001, 'colsample_bylevel': 1.0, 'gamma': 1000.0, 'score': 49.555650353431702}
{'eps': 1.0, 'neg_grad': -0.10000000000000001, 'max_depth':